
# Bees Algorithm (BA) for 0–1 Knapsack — All-in-One Notebook

This notebook contains:
- Parsers for common datasets (Burkardt, OR-Library WEING*, Pisinger knapPI_*, simple CSV).
- BA solver with feasibility repair and statistics tracking.
- Greedy and DP-exact baselines.
- Reproducible single-run and batch-run helpers.
- Plotting utilities for convergence.


In [ ]:

from __future__ import annotations
import random, re, os, json
from dataclasses import dataclass, field
from typing import List, Tuple, Optional, Dict
import math
import pandas as pd
import matplotlib.pyplot as plt


In [ ]:

@dataclass
class KnapsackInstance:
    capacity: int
    weights: List[int]
    values: List[int]
    def n(self) -> int:
        return len(self.weights)

def value_and_weight(bitvec: List[int], inst: KnapsackInstance) -> Tuple[int, int]:
    w = v = 0
    for b, wi, vi in zip(bitvec, inst.weights, inst.values):
        if b:
            w += wi
            v += vi
    return v, w


## Parsers

In [ ]:

def from_lists(capacity: int, weights: List[int], values: List[int]) -> KnapsackInstance:
    assert len(weights)==len(values)
    return KnapsackInstance(capacity=int(capacity), weights=list(map(int,weights)), values=list(map(int,values)))

def parse_burkardt(folder: str, pid: str) -> KnapsackInstance:
    """
    Burkardt format: pXX_c.txt (capacity), pXX_w.txt (weights one per line), pXX_p.txt (profits).
    """
    with open(os.path.join(folder, f"{pid}_c.txt")) as f:
        cap = int(f.read().strip())
    with open(os.path.join(folder, f"{pid}_w.txt")) as f:
        weights = [int(x.strip()) for x in f if x.strip()]
    with open(os.path.join(folder, f"{pid}_p.txt")) as f:
        values = [int(x.strip()) for x in f if x.strip()]
    return from_lists(cap, weights, values)

def parse_simple_csv(path: str) -> KnapsackInstance:
    """
    CSV with headers optional. Include a first commented line like: # capacity=750
    Then lines: weight,value
    """
    cap = None
    weights, values = [], []
    with open(path) as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            if line.startswith("#"):
                m = re.search(r"capacity\s*=\s*(\d+)", line, re.I)
                if m:
                    cap = int(m.group(1))
                continue
            parts = re.split(r"[,\s;]+", line)
            if len(parts) >= 2:
                w = int(parts[0]); v = int(parts[1])
                weights.append(w); values.append(v)
    if cap is None:
        raise ValueError("Missing capacity comment line like '# capacity=750' in CSV.")
    return from_lists(cap, weights, values)

def parse_weing(path: str) -> KnapsackInstance:
    """
    Expected layout (common variant):
        n  C
        w1 w2 ... wn
        v1 v2 ... vn
    If file contains multiple instances, only the first one is read.
    """
    with open(path) as f:
        tokens = [int(t) for t in re.findall(r"-?\d+", f.read())]
    if len(tokens) < 2:
        raise ValueError("File too short.")
    n, C = tokens[0], tokens[1]
    weights = tokens[2:2+n]
    values = tokens[2+n:2+2*n]
    if len(weights) != n or len(values) != n:
        raise ValueError("Unexpected layout; please adapt parser for this file.")
    return from_lists(C, weights, values)

def parse_knapPI(path: str) -> KnapsackInstance:
    """
    Attempts to parse a single Pisinger knapPI_* instance file.
    Try common layouts: pairs (w v) or two contiguous blocks.
    """
    with open(path) as f:
        ints = [int(t) for t in re.findall(r"-?\d+", f.read())]
    if len(ints) < 2:
        raise ValueError("File too short.")
    n, C = ints[0], ints[1]
    rest = ints[2:]
    if len(rest) >= 2*n:
        # Try pairs first
        if all(True for _ in rest):
            weights = rest[0::2][:n]
            values  = rest[1::2][:n]
            if len(weights)==n and len(values)==n:
                return from_lists(C, weights, values)
        # Fallback: first n weights then n values
        weights = rest[:n]; values = rest[n:n+n]
        if len(weights)==n and len(values)==n:
            return from_lists(C, weights, values)
    raise ValueError("Unrecognized knapPI layout. Please check the dataset and adjust parser.")


## Bees Algorithm (BA)

In [ ]:

@dataclass
class BAConfig:
    n_scouts: int = 30
    n_elite_sites: int = 3
    n_best_sites: int = 8
    n_recruits_elite: int = 20
    n_recruits_best: int = 10
    n_global_random: int = 10
    ngh_init: int = 6
    ngh_decay: float = 0.98
    max_iters: int = 2000
    seed: Optional[int] = None
    repair: str = "greedy"    # "greedy" or "drop_heaviest"
    stop_no_improve: int = 200

@dataclass
class BARunStats:
    best_values: List[int] = field(default_factory=list)
    avg_fitness: List[float] = field(default_factory=list)
    iters_to_best: Optional[int] = None
    best_weight: Optional[int] = None

def random_solution(inst: KnapsackInstance, rng: random.Random) -> List[int]:
    return [rng.randint(0,1) for _ in range(inst.n())]

def repair_solution(x: List[int], inst: KnapsackInstance, rng: random.Random, method: str="greedy") -> List[int]:
    v, w = value_and_weight(x, inst)
    if w <= inst.capacity:
        return x
    idxs = [i for i,b in enumerate(x) if b==1]
    if method == "drop_heaviest":
        idxs.sort(key=lambda i: inst.weights[i], reverse=True)
        for i in idxs:
            if w <= inst.capacity: break
            if x[i]==1:
                x[i]=0; w -= inst.weights[i]
        return x
    else:
        idxs.sort(key=lambda i: inst.values[i]/inst.weights[i])
        for i in idxs:
            if w <= inst.capacity: break
            if x[i]==1:
                x[i]=0; w -= inst.weights[i]
        return x

def flip_bits(x: List[int], indices: List[int]) -> List[int]:
    y = x[:]
    for i in indices:
        y[i] = 1 - y[i]
    return y

def neighborhood_move(x: List[int], inst: KnapsackInstance, k: int, rng: random.Random, repair: str) -> List[int]:
    if k <= 0:
        k = 1
    t = rng.randint(1, min(k, inst.n()))
    idxs = rng.sample(range(inst.n()), t)
    y = flip_bits(x, idxs)
    y = repair_solution(y, inst, rng, method=repair)
    return y

def bees_algorithm(inst: KnapsackInstance, config: BAConfig) -> Tuple[List[int], BARunStats]:
    rng = random.Random(config.seed)
    population = [repair_solution(random_solution(inst, rng), inst, rng, config.repair) for _ in range(config.n_scouts)]
    fitness = [value_and_weight(x, inst)[0] if value_and_weight(x, inst)[1] <= inst.capacity else -10**9 for x in population]
    best_idx = max(range(len(population)), key=lambda i: fitness[i])
    best = population[best_idx][:]
    best_val, best_w = value_and_weight(best, inst)
    ngh = config.ngh_init

    stats = BARunStats(best_values=[best_val], avg_fitness=[sum(fitness)/len(fitness)], iters_to_best=0, best_weight=best_w)
    no_improve = 0

    for it in range(1, config.max_iters+1):
        ranked = sorted(zip(population, fitness), key=lambda t: t[1], reverse=True)
        elite_sites = [s for s,_ in ranked[:config.n_elite_sites]]
        best_sites = [s for s,_ in ranked[config.n_elite_sites:config.n_elite_sites+config.n_best_sites]]
        new_population = []

        for s in elite_sites:
            candidates = [s]
            for _ in range(config.n_recruits_elite):
                candidates.append(neighborhood_move(s, inst, int(round(ngh)), rng, config.repair))
            cand_fit = [value_and_weight(c, inst)[0] for c in candidates]
            idx = max(range(len(candidates)), key=lambda i: cand_fit[i])
            new_population.append(candidates[idx])

        for s in best_sites:
            candidates = [s]
            for _ in range(config.n_recruits_best):
                candidates.append(neighborhood_move(s, inst, int(round(ngh)), rng, config.repair))
            cand_fit = [value_and_weight(c, inst)[0] for c in candidates]
            idx = max(range(len(candidates)), key=lambda i: cand_fit[i])
            new_population.append(candidates[idx])

        for _ in range(max(0, config.n_scouts - len(new_population))):
            if rng.random() < 0.5:
                y = repair_solution(random_solution(inst, rng), inst, rng, config.repair)
            else:
                y = neighborhood_move(best, inst, int(round(ngh)), rng, config.repair)
            new_population.append(y)

        population = new_population
        fitness = [value_and_weight(x, inst)[0] for x in population]

        cur_best_idx = max(range(len(population)), key=lambda i: fitness[i])
        cur_best = population[cur_best_idx]
        cur_val, cur_w = value_and_weight(cur_best, inst)
        if cur_val > best_val:
            best = cur_best[:]
            best_val, best_w = cur_val, cur_w
            stats.iters_to_best = it
            no_improve = 0
        else:
            no_improve += 1

        stats.best_values.append(best_val)
        stats.avg_fitness.append(sum(fitness)/len(fitness))
        stats.best_weight = best_w

        ngh = max(1, ngh * config.ngh_decay)
        if no_improve >= config.stop_no_improve:
            break

    return best, stats


## Baselines: Greedy & DP exact (optional)

In [ ]:

def greedy_by_ratio(inst: KnapsackInstance) -> List[int]:
    n = inst.n()
    order = sorted(range(n), key=lambda i: inst.values[i]/inst.weights[i], reverse=True)
    x = [0]*n
    cur_w = 0
    for i in order:
        if cur_w + inst.weights[i] <= inst.capacity:
            x[i] = 1
            cur_w += inst.weights[i]
    return x

def greedy_by_value(inst: KnapsackInstance) -> List[int]:
    n = inst.n()
    order = sorted(range(n), key=lambda i: inst.values[i], reverse=True)
    x = [0]*n
    cur_w = 0
    for i in order:
        if cur_w + inst.weights[i] <= inst.capacity:
            x[i] = 1
            cur_w += inst.weights[i]
    return x

def dp_exact(inst: KnapsackInstance) -> List[int]:
    n = inst.n()
    W = inst.capacity
    if n*W > 2_000_000:
        raise RuntimeError("DP budget guard: n*W too large for this quick implementation.")
    dp = [0]*(W+1)
    keep = [[0]*(W+1) for _ in range(n)]
    for i in range(n):
        wi, vi = inst.weights[i], inst.values[i]
        for w in range(W, wi-1, -1):
            if dp[w-wi] + vi > dp[w]:
                dp[w] = dp[w-wi] + vi
                keep[i][w] = 1
    x = [0]*n
    w = W
    for i in range(n-1, -1, -1):
        if keep[i][w] == 1:
            x[i] = 1
            w -= inst.weights[i]
    return x


## Helpers: Single run, Batch run, and Plotting

In [ ]:

def run_single(inst: KnapsackInstance, cfg: BAConfig, save_convergence_csv: Optional[str]=None, try_dp: bool=False):
    best, stats = bees_algorithm(inst, cfg)
    best_val, best_w = value_and_weight(best, inst)

    g = greedy_by_ratio(inst)
    g_val, g_w = value_and_weight(g, inst)

    print(f"n={inst.n()} W={inst.capacity}")
    print(f"BA best value: {best_val}  weight used: {best_w}")
    print(f"Greedy value:  {g_val}  weight used: {g_w}")
    if try_dp:
        try:
            x = dp_exact(inst)
            dv, dw = value_and_weight(x, inst)
            print(f"DP exact value: {dv}  weight used: {dw}")
        except Exception as e:
            print("DP exact skipped:", e)

    if save_convergence_csv:
        df = pd.DataFrame({
            "iter": list(range(len(stats.best_values))),
            "best_value": stats.best_values,
            "avg_fitness": stats.avg_fitness
        })
        df.to_csv(save_convergence_csv, index=False)
        print("Saved convergence to", save_convergence_csv)
    return best, stats

def plot_convergence(csv_path: str, out_path: Optional[str]=None):
    df = pd.read_csv(csv_path)
    plt.figure()
    plt.plot(df["iter"], df["best_value"], label="Best value")
    plt.plot(df["iter"], df["avg_fitness"], label="Avg population fitness")
    plt.xlabel("Iteration"); plt.ylabel("Value"); plt.legend(); plt.tight_layout()
    if out_path:
        plt.savefig(out_path, dpi=180)
        print("Saved plot to", out_path)
    else:
        plt.show()

def batch_runs(instances: List[Tuple[str, KnapsackInstance]], base_cfg: BAConfig, seeds: int=10) -> pd.DataFrame:
    rows = []
    for name, inst in instances:
        for s in range(seeds):
            cfg = BAConfig(**{**base_cfg.__dict__, "seed": s})
            sol, stats = bees_algorithm(inst, cfg)
            val, w = value_and_weight(sol, inst)
            rows.append({
                "instance": name,
                "n": inst.n(),
                "W": inst.capacity,
                "seed": s,
                "best_value": val,
                "weight_used": w,
                "iters_to_best": stats.iters_to_best,
                "avg_pop_final": stats.avg_fitness[-1]
            })
    return pd.DataFrame(rows)


## Quick sanity check on a tiny CSV dataset

In [ ]:

# Create a tiny CSV in the working directory for a quick run
toy_path = "toy_knap.csv"
with open(toy_path, "w") as f:
    f.write("# capacity=50\n10,60\n20,100\n33,120\n7,22\n5,20\n")

inst = parse_simple_csv(toy_path)
cfg = BAConfig(max_iters=400, seed=0)
best, stats = run_single(inst, cfg, save_convergence_csv="toy_conv.csv", try_dp=True)
plot_convergence("toy_conv.csv")  # display inline
